## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
%load_ext line_profiler

In [3]:
# 
import numpy as np

A = np.random.randint(0, 1000, size=1_000_000)
A

array([161, 520, 666, ..., 783, 494, 449])

In [4]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi=ai+100
        acc+=bi
        cnt+=1
    return acc/cnt

In [5]:
def f2(A):
    acc = 0
    for ai in A:
        bi=ai+100
        acc+=bi
    return acc/len(A)

In [6]:
def f3(A):
    return sum(A)/len(A) + 100

In [7]:
def f4(A):
    return A.mean()+100

In [8]:
# проверка времени выполнения функции

# %timeit f1(A) 
# %timeit f2(A)
# %timeit f3(A)
# %timeit f4(A)


In [9]:
# %lprun -f f2 f2(A)

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [10]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

In [11]:
%timeit g(df)

661 ms ± 35.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%lprun -f g g(df)

Timer unit: 1e-06 s

Total time: 0.647601 s
File: <ipython-input-10-e94296ffa06f>
Function: g at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def g(df):
    10         1          3.0      3.0      0.0      letters = ['a', 'b', 'c', 'd', 'e']
    11         1          1.0      1.0      0.0      dfs = []
    12         6          5.0      0.8      0.0      for letter in letters:
    13         5     628240.0 125648.0     97.0          q = df[df['key']==letter]
    14         5         11.0      2.2      0.0          dfs.append(q)
    15         1      19341.0  19341.0      3.0      return pd.concat(dfs, axis=0)

In [13]:
def g_op(df):
    return df[df["key"].isin(["a", "b", "c", "d", "e"])]


In [14]:
%timeit g_op(df)

70.2 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [15]:
import pandas as pd
recipes = pd.read_csv("./data/recipes_sample.csv", sep=",", parse_dates=['submitted'])
reviews = pd.read_csv("./data/reviews_sample.csv", sep=",", parse_dates=['date'])

In [55]:
def f1(reviews):
    """А) С использованием метода `DataFrame.iterrows` исходной таблицы"""
    acc = 0
    for _, row in reviews.iterrows():
        acc += row["rating"]
    return acc/len(reviews)


def f2(reviews):
    """Б) С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год"""
    acc, cnt = 0, 0
    for _, row in reviews.iterrows():
        if (pd.to_datetime(row["date"], format="%d.%m.%Y").year == 2010) == True:
            acc += row.loc["rating"]
            cnt += 1
    return acc/cnt


def f3(reviews):
    """В) С использованием метода `Series.mean`"""
    return reviews["rating"].mean()


print(f1(reviews))
print(f2(reviews))
print(f3(reviews))


4.410802235271832
4.4544402182900615
4.410802235271832


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [52]:
# %timeit f1(reviews)
%lprun -f f1 f1(reviews)
"""11.2 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)"""

'11.2 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)'

Timer unit: 1e-06 s

Total time: 28.3844 s
File: <ipython-input-48-0ac9f678e33d>
Function: f1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def f1(reviews):
     2                                               """А) С использованием метода `DataFrame.iterrows` исходной таблицы"""
     3         1          2.0      2.0      0.0      acc = 0
     4    126697   23827576.0    188.1     83.9      for _, row in reviews.iterrows():
     5    126696    4556804.0     36.0     16.1          acc += row["rating"]
     6         1          9.0      9.0      0.0      return acc/len(reviews)

In [56]:
# %timeit f2(reviews)
%lprun -f f2 f2(reviews)
"""12 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)"""

'12 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)'

Timer unit: 1e-06 s

Total time: 29.719 s
File: <ipython-input-55-c01bc0552373>
Function: f2 at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def f2(reviews):
    10                                               """Б) С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год"""
    11         1          2.0      2.0      0.0      acc, cnt = 0, 0
    12    126697   23359190.0    184.4     78.6      for _, row in reviews.iterrows():
    13    126696    5524478.0     43.6     18.6          if (pd.to_datetime(row["date"], format="%d.%m.%Y").year == 2010) == True:
    14     12094     825733.0     68.3      2.8              acc += row.loc["rating"]
    15     12094       9583.0      0.8      0.0              cnt += 1
    16         1          1.0      1.0      0.0      return acc/cnt

In [54]:
# %timeit f3(reviews)
%lprun -f f3 f3(reviews)
"""168 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)"""


'168 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)'

Timer unit: 1e-06 s

Total time: 0.000514 s
File: <ipython-input-48-0ac9f678e33d>
Function: f3 at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                           def f3(reviews):
    22                                               """В) С использованием метода `Series.mean`"""
    23         1        514.0    514.0    100.0      return reviews["rating"].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

%timeit get_word_reviews_count(reviews)

KeyboardInterrupt: 

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.
